# Higher-order integration methods

- Quadrature is a historical term which means the process of determining area, or integrals. 
- Trapezoidal rule is based on approximating the integrand $f(x)$ with straight-line segments.
- Simpson's rules uses quadratics.
- Recall: The *trapezoidal* rule is $$F_n = \left[\frac{1}{2}f(x_0) \displaystyle \sum_{i=1}^{n-1}f(x_i) + \frac{1}{2}f(x_n) \right]\Delta x$$ and *Simpson's rule* is $$F_n = \frac{1}{3} [f(x_0) + 4f(x_1) + 2f(x_2) + 4f(x_3) + \cdots + 2f(x_{n-2}) + 4f(x_{n-1}) + f(x_n)] \Delta x$$
- We create higher-order polynomial fits to $f(x)$. In general, $$\int_a^b f(x)dx \approx \displaystyle \sum_{k=1}^{n} w_k f(x_k) = F_n$$ where the $x_k$ are the positions of the sample points at which we calculate the integrand and the $w_k$ are a set of weights.
- In the trapezoidal rule the first and last weights are $\frac{1}{2}$ and the others are $1$, while Simpson's rule uses $\frac{1}{3}$ for the first and last and alternate between $\frac{4}{3}$ and $\frac{2}{3}$ for the other slices.
- Let's broaden the rules to allow us to vary not only the weights $w_k$ but also the positions $x_k$. It is possible to create an integration method that is exact for polynomials up to degree $n-1$ with $n$ equally spaced points.
- Varying the positions introduces an extra $n$ degrees of freedom, which suggests that it might be possible to create an integration rule that is exact for polynomials up to degree $2n-1$.
- This scheme is known as *Gaussian Quadrature* and produces very accurate integration results!

# Gaussian Quadrature
1. First step is to derive the integration rules for nonuniform smaple points $x_k$.
    - Use Lagrange polynomials for the interpolation, i.e., *method of interpolating polynomials* $$p_k(x) = f(x_k) \displaystyle \prod_{m=1,m\ne k}^n \frac{x-x_m}{x_k - x_m} = f(x_k)\phi_k(x)$$ such that $$p(x) = \displaystyle \sum_{k=1}{n} f(x_k)\phi_k(x)$$
    - Recall, this polynomial fit is unique.
    - Calculate an approximation to the integral, $$\int_a^b f(x)dx \approx \int_a^b p(x)dx = \displaystyle \sum_{k=1}^n f(x_k) \int_a^b \phi_k (x)dx$$
    - From above, note that the weights are $$w_k = \int_a^b \phi_k(x)dx$$
2. Second, we will choose a particular set of nonuniform points to optimize the results.
    - Without proof, claim it is possible to choose our $n$ points so that our integration rule is exact for all polynomial integrands up to and including polynomials of degree $2n-1$ (proof is beyond our scope).
    - To get an integration rule accurate up to the highest possible degree of $2n-1$, the sample points should be chosen to coincide with the zeros of teh $n$th degree Legendre polynomial $p_n(x)$.
    - For historical reasons it is necessary to map the interval to $-1 \le x \le 1$, so that $$w_k = \int_{-1}^1 \phi_k(x)dx$$ The mapping rule is $$x'_k = \frac{1}{2}(b-a)x_k + \frac{1}{2}(b+a)$$ and $$w'_k = \frac{1}{2}(b-a)w_k$$
    - The rescaled weights are given by $$w_k = \left[\frac{2}{(1-x^2)} \left(\frac{dp_n(x)}{dx} \right)^{-2}\right]$$

## Example
- Integrate $$\int_0^2(x^4-2x+1)dx$$ using only 3 points! The answer is 4.4

In [1]:
import numpy as np

######################################################################
#
# Functions to calculate integration points and weights for Gaussian
# quadrature
#
# x,w = gaussxw(N) returns integration points x and integration
#           weights w such that sum_i w[i]*f(x[i]) is the Nth-order
#           Gaussian approximation to the integral int_{-1}^1 f(x) dx
# x,w = gaussxwab(N,a,b) returns integration points and weights
#           mapped to the interval [a,b], so that sum_i w[i]*f(x[i])
#           is the Nth-order Gaussian approximation to the integral
#           int_a^b f(x) dx
#
# This code finds the zeros of the nth Legendre polynomial using
# Newton's method, starting from the approximation given in Abramowitz
# and Stegun 22.16.6. The Legendre polynomial itself is evaluated
# using the recurrence relation given in Abramowitz and Stegun
# 22.7.10. The function has been checked against other sources for
# values of N up to 1000. It is compatible with version 2 and version
# 3 of Python.
#
# Written by Mark Newman <mejn@umich.edu>, June 4, 2011
# You may use, share, or modify this file freely
#
######################################################################

from numpy import ones,copy,cos,tan,pi,linspace

def gaussxw(N):

    # Initial approximation to roots of the Legendre polynomial
    a = linspace(3,4*N-1,N)/(4*N+2)
    x = cos(pi*a+1/(8*N*N*tan(a)))

    # Find roots using Newton's method
    epsilon = 1e-15
    delta = 1.0
    while delta>epsilon:
        p0 = ones(N,float)
        p1 = copy(x)
        for k in range(1,N):
            p0,p1 = p1,((2*k+1)*x*p1-k*p0)/(k+1)
        dp = (N+1)*(p0-x*p1)/(1-x*x)
        dx = p1/dp
        x -= dx
        delta = max(abs(dx))

    # Calculate the weights
    w = 2*(N+1)*(N+1)/(N*N*(1-x*x)*dp*dp)

    return x,w

def gaussxwab(N,a,b):
    x,w = gaussxw(N)
    return 0.5*(b-a)*x+0.5*(b+a),0.5*(b-a)*w

def f(x):
    return x**4 - 2*x + 1

n = 3
a = 0.0
b = 2.0

x,w = gaussxwab(n,a,b)

#Perform the integration
s = np.sum(w*f(x))
print(s)

4.4000000000000075


- The program produces the answer "exactly", with just three sample points! The result is exact for integrals of polynomial functions up to and including degree $2n-1$ (5th degree for $n=3$). Here we have a 4th degree polynomial, so we expect the method to return the exact answer.